In [1]:



import http.client
import json
import pandas as pd
import time

#List of aircraft registrations
registrations = [
    # United
    "N23983", "N25982","N29981", "N24976", "N12012",
    # American
    "N874AN", "N873BB", "N872AN", "N838AA", "N837AN",
    # All Nippon
    "JA933A", "JA932A", "JA928A", "JA897A", "JA898A",
    # Qatar
    "A7-BHI", "A7-BHF", "A7-BHD", "A7-BHA", "A7-BDD"
]
# registrations = ["N23983"]

# Set up the API connection
conn = http.client.HTTPSConnection("flight-radar1.p.rapidapi.com")

# Define headers with your RapidAPI key
headers = {
    'x-rapidapi-key': "e8fb405032msh03df0c0e47a6579p10ca69jsncdfb00f0d8b7",  # Replace with your actual RapidAPI key
    'x-rapidapi-host': "flight-radar1.p.rapidapi.com"
}

# Initialize an empty list to store DataFrames
all_dfs = []

# Define desired columns
# columns = [
#     "identification.number.default",
#     "identification.callsign",
#     "status.text",
#     "airport.origin.name",
#     "airport.destination.name",
#     "time.scheduled.departure",
#     "time.scheduled.arrival",
#     "time.real.departure",
#     "time.real.arrival",
#     "time.estimated.departure",
#     "time.estimated.arrival",
#     "aircraft.registration"  # Added to track the registration
# ]


columns = [
        "aircraft.registration",  # Flight Number
        "aircraft.model.code",
        "time.real.departure",       # Callsign
        "time.real.arrival",                   # Status
        "status.generic.status.text",           # Origin Airport
        "airline.code.icao",      # Destination Airport
        "airline.name",
        "airport.destination.position.latitude",
        "airport.destination.position.longitude"# Scheduled Departure (Unix timestamp)
]


# Loop through each registration
for reg in registrations:
    try:
        # Make the API request for the current registration
        conn.request("GET", f"/flights/get-more-info?query={reg}&fetchBy=reg&page=1&limit=100", headers=headers)
        
        # Get the response
        res = conn.getresponse()
        
        # Check if the request was successful
        if res.status == 200:
            data = res.read()
            json_data = json.loads(data.decode("utf-8"))
            
            # Extract the 'data' list from the response
            flight_data = json_data.get("result", {}).get("response", {}).get("data", [])
            
            if flight_data:
                # Convert to DataFrame
                df = pd.json_normalize(flight_data)
                
                # Add the registration to the DataFrame (in case it's not included or to ensure clarity)
                df["aircraft.registration"] = reg
                
                # Filter to desired columns (only include columns that exist)
                available_columns = [col for col in columns if col in df.columns]
                df = df[available_columns]
                
                # Append to the list of DataFrames
                all_dfs.append(df)
                
                print(f"Successfully fetched data for registration: {reg}")
            else:
                print(f"No flight data found for registration: {reg}")
        else:
            print(f"Error: API request failed for {reg} with status code {res.status}")
        
        # Add a delay to respect API rate limits (adjust as needed)
        time.sleep(1)
        
    except json.JSONDecodeError:
        print(f"Error: Failed to decode JSON response for {reg}")
    except Exception as e:
        print(f"Error for {reg}: {str(e)}")

# Close the connection
conn.close()

# Combine all DataFrames
if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=True)
    
    # Rename columns for clarity
#     final_df.columns = [
#         "Flight Number",
#         "Callsign",
#         "Status",
#         "Origin Airport",
#         "Destination Airport",
#         "Scheduled Departure",
#         "Scheduled Arrival",
#         "Actual Departure",
#         "Actual Arrival",
#         "Estimated Departure",
#         "Estimated Arrival",
#         "Aircraft Registration"
#     ][-len(final_df.columns):]  # Adjust based on available columns
    
    final_df.columns = [
        "tail_number",
        "aircraft_model",
        "real_departure",
        "real_arrival",
        "status",
        "icao_code",
        "airline_name",
        "destination_airport_lat",
        "destination_airport_lat"
    ][-len(final_df.columns):]  # Adjust based on available columns
    
    # Convert Unix timestamps to human-readable datetime
#     time_columns = [
#         "Scheduled Departure",
#         "Scheduled Arrival",
#         "Actual Departure",
#         "Actual Arrival",
#         "Estimated Departure",
#         "Estimated Arrival"
#     ]
    
    time_columns = [
        "real_departure",
        "real_arrival",
    ]
    

    for col in time_columns:
        if col in final_df.columns:
            final_df[col] = pd.to_datetime(final_df[col], unit="s", errors="coerce")
    
    # Display the DataFrame
    print("\nFinal DataFrame:")
    print(final_df.head())
    
    # Save to CSV
    final_df.to_csv("all_aircraft_flight_data_test12.csv", index=False)
    print("\nData saved to 'all_aircraft_flight_data_test12.csv'")
else:
    print("No data was collected.")


Successfully fetched data for registration: N23983
Successfully fetched data for registration: N25982
Successfully fetched data for registration: N29981
Successfully fetched data for registration: N24976
Successfully fetched data for registration: N12012
Successfully fetched data for registration: N874AN
Successfully fetched data for registration: N873BB
Successfully fetched data for registration: N872AN
Successfully fetched data for registration: N838AA
Successfully fetched data for registration: N837AN
Successfully fetched data for registration: JA933A
Successfully fetched data for registration: JA932A
Successfully fetched data for registration: JA928A
Successfully fetched data for registration: JA897A
Successfully fetched data for registration: JA898A
Successfully fetched data for registration: A7-BHI
Successfully fetched data for registration: A7-BHF
Successfully fetched data for registration: A7-BHD
Successfully fetched data for registration: A7-BHA
Successfully fetched data for r